In [1]:
import pandas as pd
import numpy as np

### LASSO

In [2]:
df_train = pd.read_csv('../data/train_con_features.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('../data/test_con_features.csv', index_col='Unnamed: 0')
df_train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,piscina,escuelascercanas,centroscomercialescercanos,precio,cantidad_amenities,cantidad_servicios,relacion_metros,metros_por_habitacion,metros_por_bano,metros_por_amenities
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116114,2.0,1.0,...,0.0,0.0,0.0,2273000.0,0.0,0.0,1.000000,40.000000,40.0,0.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,...,0.0,1.0,1.0,3600000.0,0.0,2.0,1.488889,89.333333,134.0,0.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,...,0.0,0.0,0.0,1200000.0,0.0,0.0,0.867470,48.000000,72.0,0.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,...,0.0,1.0,1.0,650000.0,0.0,2.0,0.940299,31.500000,63.0,0.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,...,0.0,0.0,0.0,1150000.0,0.0,0.0,1.000000,47.500000,95.0,0.0


In [3]:
columnas_numericas = ['antiguedad', 'habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina', 
                      'escuelascercanas', 'centroscomercialescercanos', 'cantidad_amenities', 'cantidad_servicios', 
                      'relacion_metros', 'metros_por_amenities']

columnas_numericas_y_precio = ['antiguedad', 'habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina', 
                               'escuelascercanas', 'centroscomercialescercanos', 'cantidad_amenities', 'cantidad_servicios', 
                               'relacion_metros', 'metros_por_amenities']

#'metros_por_bano', 'metros_por_habitacion'

columnas_numericas_y_precio.append('precio')

In [4]:
X = df_train[columnas_numericas_y_precio].drop(columns=['precio']).values
y = df_train['precio'].values

In [5]:
# Métrica de evaluación
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(216000, 12) (216000,)
(24000, 12) (24000,)


In [7]:
from sklearn import linear_model

def tunning_alpha (Datos_train, val_train, Datos_test, un_alpha):
    lasso = linear_model.Lasso(alpha=un_alpha,positive=True)
    lasso_fit = lasso.fit(Datos_train,val_train) 
    lasso_pred = lasso_fit.predict(Datos_test)
    return (lasso_fit, lasso_pred)

In [9]:
alphas = np.random.rand(9)
display(alphas)

array([0.26502229, 0.43952492, 0.77583207, 0.724954  , 0.12254359,
       0.691444  , 0.82198532, 0.27646127, 0.05571908])

In [10]:
#alphas = [0.02, 0.024, 0.025, 0.026, 0.03]

for un_alpha in alphas:
    lasso_fit_alpha, lasso_pred_alpha = tunning_alpha(X_train, y_train, X_test, un_alpha)
    lasso_rmsle = RMSLE(y_test, lasso_pred_alpha)
    lasso_rmsle_train = RMSLE(y_train, lasso_fit_alpha.predict(X_train))
    print(f"RMSLE Lasso (train): {lasso_rmsle_train:.5f}", "con alpha =", un_alpha)
    print(f"RMSLE Lasso: {lasso_rmsle:.5f}", "con alpha =", un_alpha)
    print("------------------------------")

RMSLE Lasso (train): 0.78031 con alpha = 0.26502228565868724
RMSLE Lasso: 0.78558 con alpha = 0.26502228565868724
------------------------------
RMSLE Lasso (train): 0.78031 con alpha = 0.43952492455801195
RMSLE Lasso: 0.78558 con alpha = 0.43952492455801195
------------------------------
RMSLE Lasso (train): 0.78031 con alpha = 0.7758320676789286
RMSLE Lasso: 0.78558 con alpha = 0.7758320676789286
------------------------------
RMSLE Lasso (train): 0.78031 con alpha = 0.7249540037035691
RMSLE Lasso: 0.78558 con alpha = 0.7249540037035691
------------------------------
RMSLE Lasso (train): 0.78031 con alpha = 0.1225435932129052
RMSLE Lasso: 0.78558 con alpha = 0.1225435932129052
------------------------------
RMSLE Lasso (train): 0.78031 con alpha = 0.6914440012624173
RMSLE Lasso: 0.78558 con alpha = 0.6914440012624173
------------------------------
RMSLE Lasso (train): 0.78031 con alpha = 0.8219853206411967
RMSLE Lasso: 0.78558 con alpha = 0.8219853206411967
--------------------------

Cambiando el parámetro ***alpha*** practicamente los mismos resultados

###### Con 'metros_por_habitacion' y 'metros_por_banos' tira error el calculo del logaritmo
###### C:\Users\maxi_\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log. This is separate from the ipykernel package so we can avoid doing imports until

In [12]:
#lasso_rmsle = RMSLE(y_test, lasso_pred)
#lasso_rmsle_train = RMSLE(y_train, lasso_fit.predict(X_train))
#print(f"RMSLE Lasso (train): {lasso_rmsle_train:.5f}")
#print(f"RMSLE Lasso: {lasso_rmsle:.5f}")

In [13]:
df_train_sumbit = df_train[columnas_numericas_y_precio]
df_test_submit = df_test[columnas_numericas]
lasso_pred_sumbit = linear_model.Lasso(alpha=0.1,positive=True)\
                        .fit(df_train_sumbit[columnas_numericas], df_train_sumbit['precio'])\
                        .predict(df_test_submit[columnas_numericas])

In [14]:
resultado_submit = pd.DataFrame(index=df_test.index)
resultado_submit['id'] = df_test['id']
resultado_submit['target'] = lasso_pred_sumbit
display(resultado_submit.head())
resultado_submit.to_csv('../data/sumbitLasso.csv',index=False)

,id,target
0,4941,3.173651e+06
1,51775,1.314068e+06
2,115253,1.462336e+06
3,299321,2.199259e+06
4,173570,1.749755e+06


#### Score en Kaggle (features => score):
 - *['habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos'] =>* ***1433282.27514*** con alpha = 0.1
 - *['antiguedad', 'habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina','escuelascercanas', 'centroscomercialescercanos', 'cantidad_amenities', 'cantidad_servicios', 'relacion_metros', 'metros_por_amenities'] =>* ***1400340.32238*** con alpha = 0.1